In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os
import joblib

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from collections import Counter
from datetime import date 
import datetime


In [6]:
devices_df = pd.read_csv('devices.csv')
readings_df = pd.read_csv('sampled_readings.csv')
reading_types_df = pd.read_csv('reading_types.csv')

### Outlier Detection Using IQR

In [7]:
def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.nanpercentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.nanpercentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        print("First Quartertile:", Q1, ". Third Quartile: ", Q3, ".Interquartile Range: ", IQR)
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v >= n )
    
    return multiple_outliers   

In [8]:
df = readings_df 
for k, v in readings_df.groupby('value_type_id'):
    outliers = detect_outliers(v, 1, ['value'])
    df = df.drop(outliers, axis = 0) 

df.info()

First Quartertile: 428.0 . Third Quartile:  564.8 .Interquartile Range:  136.79999999999995
First Quartertile: 0.0 . Third Quartile:  1.7 .Interquartile Range:  1.7
First Quartertile: 31.0 . Third Quartile:  249.0 .Interquartile Range:  218.0
First Quartertile: 0.0 . Third Quartile:  0.0 .Interquartile Range:  0.0
First Quartertile: 0.0 . Third Quartile:  0.2 .Interquartile Range:  0.2
First Quartertile: 0.0 . Third Quartile:  0.1 .Interquartile Range:  0.1
First Quartertile: 3.8 . Third Quartile:  13.2 .Interquartile Range:  9.399999999999999
First Quartertile: 0.0 . Third Quartile:  31.7 .Interquartile Range:  31.7
First Quartertile: 18.6 . Third Quartile:  20.0 .Interquartile Range:  1.3999999999999986
First Quartertile: 0.0 . Third Quartile:  23.0 .Interquartile Range:  23.0
First Quartertile: 20.4 . Third Quartile:  23.6 .Interquartile Range:  3.200000000000003
First Quartertile: 27.9 . Third Quartile:  45.1 .Interquartile Range:  17.200000000000003
<class 'pandas.core.frame.DataF

### Merging devices with sampled readings
Since devices in the same building_id are situated in the same environment we should expect that they share similar IAQ. There may be differences depending on the # of people in different rooms but we will hypothesize that the difference is minimal. Here we map device_ids to buildings to group all devices by building_id

In [9]:
#merging devices with sampled readings

df = pd.merge(df, devices_df, on='device_id', how='inner')
df = df.drop('device_id', axis = 1)

### Have aggregate value_types with the same hour
Since the data is not given in consistent time-steps we will use downsampling to aggregate data points for 5 minute time-steps. We will partition the data based on value_type_id as well as building_id

In [10]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.floor('5min')

aggregate_function = {'value': 'mean'}
df = df.groupby(['building_id', 'date', 'value_type_id']).agg(aggregate_function)


#pivot table so that value_type_id is a column 
df = pd.pivot_table(df, values = 'value', index = ['date', 'building_id'], columns = 'value_type_id').reset_index()  
df = df.rename_axis(None).rename_axis(None, axis=1)
df.columns = df.columns.map(str)

### Interpolating Small Gaps
For small gaps (15 minutes) in data we will use interpolation to predict missing values 

In [12]:
# df = df.sort_values(by = 'date')

# def my_interp(x):
#     if x.notnull().sum() > 1:
#         return x.interpolate(method='nearest').ffill().bfill()
#     else:
#         return x.ffill().bfill()
interp = pd.DataFrame()
for building, df1 in df.groupby('building_id'):
        df1 = df1.sort_values(by = 'date')
        # resampled = df.resample('60min', on = 'date', label = 'left').mean() 
        # resampled ['date'] = resampled.index.values
        for i in range (1, 13): 
                cnt = df1[str(i)].count()
                if cnt != 0: 
                        df1[str(i)] = df1[str(i)].interpolate(method='spline', order = min(cnt - 1, 3), limit = 3, axis=0)
                else:  
                        df1[str(i)] = df1[str(i)].fillna(0)
        interp = pd.concat([interp, df1], ignore_index = True)
# df.info()
# df = df.sort_values(by = 'date')
# print(df.head(100))
# # resampled_df.to_csv('googoogagag.csv')

TypeError: unsupported operand type(s) for -: 'method' and 'int'

In [ ]:
df = interp

# Feature Generation

### Working Hour
Since IAQ most likely decreases off-work hours or when there is a lack of personell we will add features to determine working hours and weekends

In [ ]:
# hour_mapping (1 if between 8am and 6pm)

# readings_df['date'] = pd.to_datetime(readings_df['date'])

df['work_hours'] = df['date'].dt.hour.between(8, 18)
df['work_hours'].map({True: 1, False: 0})


In [ ]:
# day of week mapping (1 weekday, 0 weekend)

df['day type'] = df['date'].dt.dayofweek.map({
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 0, 
    6: 0
})


In [ ]:
# season mapping, use or not depending on seasonality dicky-fuller test

df['season'] = df['date'].dt.month.map({
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Fall',
    10: 'Fall',
    11: 'Fall',
    12: 'Winter'
})

season_encoder = pd.get_dummies(df['season'])
df = df.join(season_encoder)
df = df.drop('season', axis = 1)

In [ ]:
def get_trimester_day(row): 
    dt = (row['date']).date()
    if 3 <= dt.month <= 5:
        return (dt - date(year=dt.year, month=3, day=1)) # Spring
    elif 6 <= dt.month <= 8:
        return (dt - date(year=dt.year, month=6, day=1))  # Summer
    elif 9 <= dt.month <= 11:
        return (dt - date(year=dt.year, month=9, day=1))  # Autumn
    else:
        if(dt.month == 12): 
            return (dt - date(year=dt.year, month=12, day=1))
        return (dt - date(year=dt.year - 1, month=12, day=1))  # Winter
    
df['trimester_day'] = df.apply(get_trimester_day, axis = 1)
df['trimester_day'] = df['trimester_day'].dt.days


### Building Encoder

In [ ]:
# final_df = resampled_df 
# building_encoder = pd.get_dummies(final_df['building_id'], prefix  = 'building')
# final_df = final_df.join(building_encoder)
# final_df = final_df.drop('building_id', axis = 1) 

# device_encoder = pd.get_dummies(mergedDf['device_id'], prefix = 'device')
# mergedDf = mergedDf.join(device_encoder) don't know if this matters as much 

#### IMPORTANT: uncomment the line highlighted if you do not have preprocessed.csv, use this in the model.ipynb (so we stop working on same file)

In [ ]:
df.to_csv('preprocessed_lstm.csv') # <-- COMMENT THIS OUT IF YOU DON'T HAVE preprocessd.csv yet 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832737 entries, 0 to 1832736
Data columns (total 21 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   building_id    int64         
 2   1              float64       
 3   2              float64       
 4   3              float64       
 5   4              float64       
 6   5              float64       
 7   6              float64       
 8   7              float64       
 9   8              float64       
 10  9              float64       
 11  10             float64       
 12  11             float64       
 13  12             float64       
 14  work_hours     bool          
 15  day type       int64         
 16  Fall           bool          
 17  Spring         bool          
 18  Summer         bool          
 19  Winter         bool          
 20  trimester_day  int64         
dtypes: bool(5), datetime64[ns](1), float64(12), int64(3)
memory usage: 232.5 MB
N

In [ ]:
print(df.info(verbose = True, show_counts = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832737 entries, 0 to 1832736
Data columns (total 21 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   date           1832737 non-null  datetime64[ns]
 1   building_id    1832737 non-null  int64         
 2   1              1043716 non-null  float64       
 3   2              1074519 non-null  float64       
 4   3              1024387 non-null  float64       
 5   4              943762 non-null   float64       
 6   5              971948 non-null   float64       
 7   6              983870 non-null   float64       
 8   7              1043221 non-null  float64       
 9   8              440684 non-null   float64       
 10  9              1006545 non-null  float64       
 11  10             598425 non-null   float64       
 12  11             1017428 non-null  float64       
 13  12             1072212 non-null  float64       
 14  work_hours     1832737 non-null  b